# <span style="color:brown"> **Hands-on Deep Learning – Image Classification**

_This hands-on was originally created by Thomas Grenier (TensorFlow) and Fabien Millioz (PyTorch), CREATIS.

thomas.grenier@creatis.insa-lyon.fr,
michael.sdika@creatis.insa-lyon.fr,
olivier.bernard@creatis.insa-lyon.fr,
odyssee.merveille@creatis.insa-lyon.fr,
fabien.millioz@creatis.insa-lyon.fr

# <span style="color:brown"> **Introduction on data**

This hands-on is computer scientist-oriented more than application-oriented.

And so, this practice focuses on a meaningless toy example inspired from the MNIST manuscript numbers classification challenge that is considered as the 'hello world' example for some machine learning and almost all neural networks.

MR images come from [IXI Dataset](https://brain-development.org/ixi-dataset/)

Here, we have to recognize whether a brain slice image is axial, sagittal or coronal and comes from MRI T1w, MRI T2w and Proton Density MR (PD). So, this is a classification problem with 9 classes, as follows:
   
<img src="./figures/Figure1_SagittalAxialCoronal_small.png" alt="SagittalAxialCoronal" style="width: 70%"/>

Amount of images of each class for train and test datasets is :  
- Total images for training:
   - PD-A (PD Axial)    : 2701
   - PD-C (PD Coronal)  : 2723
   - PD-S (PD Sagittal) : 2698
   - T1-A (T1 Axial)    : 2715
   - T1-C (T1 Coronal)  : 2758
   - T1-S (T1 Sagittal) : 2711
   - T2-A (T2 Axial)    : 2696
   - T2-C (T2 Coronal)  : 2717
   - T2-S (T2 Sagittal) : 2699
- Total images for testing:
   - PD-A (PD Axial)     : 689
   - PD-C (PD Coronal)   : 696
   - PD-S (PD Sagittal)  : 693
   - T1-A (T1 Axial)     : 679
   - T1-C (T1 Coronal)   : 685
   - T1-S (T1 Sagittal)  : 676
   - T2-A (T2 Axial)     : 684
   - T2-C (T2 Coronal)   : 690
   - T2-S (T2 Sagittal)  : 686

We have enough data to test classification approaches such as random forest, MLP and CNN.

> Let's go!



# <span style="color:brown"> **A - Setup**

### A1 - First, import a few common modules, ensure MatplotLib plots figures inline, and functions for converting data and displaying reports:

In [ ]:
# Common imports
import os
import glob

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np

!pip install tqdm
from tqdm import tqdm

!pip install scikit-learn
from sklearn import datasets, metrics

# if trouble with libGL consider to execute this line in a shell: apt-get install python3-opencv
# !pip install opencv-python
import cv2


### A2- We import TensorFlow 2

In [ ]:
import tensorflow as tf
print(tf.__version__)

> _**if no error occurs, your working environment is ok and you can go to next part,
> else ... call an assistant for help!**_

# <span style="color:brown"> **B - Read the data**

<span style="color:red">
    

We first download the data and then deflate them.

In [ ]:
!rm dlss21_ho1_data.tar.gz
!wget "https://gitlab.in2p3.fr/thomas.grenier/tp1ss_classification/-/raw/master/dlss21_ho1_data.tar.gz"
#!wget "https://www.creatis.insa-lyon.fr/~grenier/wp-content/uploads/teaching/DeepLearning/dlss21_ho1_data.tar.gz"
!tar xzf dlss21_ho1_data.tar.gz

### <span style="color:brown"> **B1 - Open files and convert to numpy / tensorflow** 

Now start by reading _png_ files stored in the data train and test directories. 
Note that the filename of the _png_ image describes which view (Axial, Coronal, Sagital) and modality (T1w, T2w and PD) correspond to the image.

These _png_ correspond to slices extracted from 3D volumes of the [IXI dataset](https://brain-development.org/ixi-dataset/). In order to reduce the data variability (and simplify the slice extraction scripts), all MRI volumes were registered on a reference image using [elastix](http://elastix.isi.uu.nl/) with an affine transform. 

Thus, origin, spacing, orientation and number of pixels per dimension of all volumes are the same and therefore will not need to be taken into account systematically by the learning scripts (using SimpleITK, MedPy, nibabel...).

Then, slices are extracted and the window/level was adapted (using [ITK](https://itk.org/)).

Finally, they were resized then padded/cropped to 64x64 using [ImageMagick](https://www.imagemagick.org/).

Image files in __dlss21_ho1_data/train/__  are organized as follow:
  - dlss21_ho1_data/train/PD-A/002_116_PD-A.png
                              /002_42_PD-A.png
                              /002_44_PD-A.png
                              /002_53_PD-A.png
                               ...
  - dlss21_ho1_data/train/PD-C/002_152_PD-C.png
                              /002_175_PD-C.png
                              /002_177_PD-C.png
                              /002_198_PD-C.png
                               ...
  - dlss21_ho1_data/train/PD-S/...
  - dlss21_ho1_data/train/T1-A/...
  - dlss21_ho1_data/train/T1-C/...
  - dlss21_ho1_data/train/T1-S/...
  - dlss21_ho1_data/train/T2-A/...
  - dlss21_ho1_data/train/T2-C/...
  - dlss21_ho1_data/train/T2-S/...

And the same for __dlss21_ho1_data/test/__ with different subjects than ones present in the train set.


> The next cells will open all images and create three datasets containing the images and the corresponding labels.


Some important values and initializations:

In [ ]:
desired_height = 64
desired_width  = 64

input_train_path = os.getcwd()+'/dlss21_ho1_data/train'           
input_test_path = os.getcwd()+'/dlss21_ho1_data/test'            
print(f"input train path : {input_train_path}")
print(f"input test path  : {input_test_path}")

class_names = ['PD-A', 'PD-C', 'PD-S', 'T1-A', 'T1-C', 'T1-S', 'T2-A', 'T2-C', 'T2-S']

In [ ]:
# first list files for the train and validation sets.
train_val_image_file_list = []
train_val_label_list = []

for i in range(len(class_names)):
    for filename in glob.iglob( os.path.join(input_train_path,class_names[i]) + '/**/*.png', recursive=True):
        # extract patient number and slice
        train_val_image_file_list.append(filename)
        train_val_label_list.append( i )     

print(len(train_val_image_file_list), len(train_val_label_list))
print(train_val_image_file_list[0])

In [ ]:
# Random permutation
permutation = np.random.permutation( len(train_val_label_list) )
train_val_images_files=[train_val_image_file_list[i] for i in permutation]
train_val_labels=[train_val_label_list[i] for i in permutation]

In [ ]:
# we split the train set in two parts : train and validation
VALIDATION_RATIO = 0.2
nb_train = int( len(train_val_label_list) * (1 - VALIDATION_RATIO) )
# list images for both datasets
train_images_files = train_val_images_files[:nb_train]
train_labels =train_val_labels[:nb_train]

val_images_files = train_val_images_files[nb_train:]
val_labels = train_val_labels[nb_train:]

# for labels also convert them to numpy
y_train = np.asarray(train_labels)
y_val = np.asarray(val_labels)

print( train_val_images_files[0])

In [ ]:
# Reading images 
def ReadImages(images_files):
    X = []
    for index in tqdm(range(len(images_files))):
        image_read = cv2.imread(images_files[index], cv2.IMREAD_COLOR)
        image_read = cv2.resize(image_read, dsize = (desired_width, desired_height), interpolation = cv2.INTER_LINEAR)
        X.append(image_read)
    X = np.asarray(X, dtype=np.uint8)
    return X

In [ ]:
# Read images
X_train_rgb = ReadImages(train_images_files)
X_val_rgb   = ReadImages(val_images_files)

### <span style="color:brown"> **B2 - Check the output data size and type**
We recommend to check as often as needed the shape and type of data.

Don't go further if you can't see what is inside array and tensor!

In [ ]:
# Show information
print(" Shape : " , X_train_rgb.shape, X_val_rgb.shape)
print(" Type  : %s  %s"%(X_train_rgb.dtype, X_val_rgb.dtype))
print(" Max   : %d  %d"%(X_train_rgb.max(), X_val_rgb.max()))
print(" Min   : %d  %d"%(X_train_rgb.min(), X_val_rgb.min()))

print(" Labels Shape : " , y_train.shape, y_val.shape)

### <span style="color:brown"> **B3 - Displaying images**

In [ ]:
def plot_image(image):
    plt.imshow(image, aspect="equal", cmap="gray", interpolation="nearest")
    plt.axis("off")    

In [ ]:
nb_x = 5
nb_y = 5
plt.figure(figsize=(15,15*nb_x/nb_y), dpi=100)
for i in range(1, nb_x * nb_y +1):
    plt.subplot(nb_x, nb_y ,i)
    plot_image( X_train_rgb[i-1] )
    plt.title(class_names[y_train[i-1]])

### <span style="color:brown"> **B4 - Test set**

In [ ]:
# doing the same for test set
test_image_file_list = []
test_label_list = []
           
for i in range(len(class_names)):
    for filename in glob.iglob( os.path.join(input_test_path,class_names[i]) + '/**/*.png', recursive=True):
        # extract patient number and slice
        test_image_file_list.append(filename)
        test_label_list.append( i )            

print(len(test_image_file_list), len(test_label_list))

X_test_rgb = ReadImages(test_image_file_list)
y_test = np.asarray(test_label_list)

print(" Shape : " , X_test_rgb.shape, y_test.shape)
print(" Type  : %s  %s"%(X_test_rgb.dtype, y_test.dtype))
print(" Max   : %d  %d"%(X_test_rgb.max(), y_test.max()))
print(" Min   : %d  %d"%(X_test_rgb.min(), y_test.min()))

nb_x = 5
nb_y = 5
plt.figure(figsize=(15,15*nb_x/nb_y), dpi=100)
for i in range(1, nb_x * nb_y +1):
    plt.subplot(nb_x, nb_y ,i)
    plot_image( X_test_rgb[i-1] )
    plt.title(class_names[y_test[i-1]])
# here we have not shuffle filenames :p

Verify shapes that should look like ( batch_size, desired_height, desired_width, 3 ) and (batch_size, )

### <span style="color:brown"> **B5 - Convert the data shape and type**
The _png_ files are RGB, we convert them to gray images for each set (train, validation and test) in order to be compatible with all tested approaches.

We also divide intensity values by 255 to bound range in [0;1]


In [ ]:
# converting the three data set : rgb to gray scale images, then normalization

X_train = X_train_rgb[:,:,:,1].astype(np.float32)/255
X_train = np.expand_dims(X_train,-1)

X_val = X_val_rgb[:,:,:,1].astype(np.float32)/255
X_val = np.expand_dims(X_val,-1)

X_test = X_test_rgb[:,:,:,1].astype(np.float32)/255
X_test = np.expand_dims(X_test,-1)

print(" Shape : ", X_train.shape, X_val.shape, X_test.shape)
print(" Type  : %s  %s %s"%(X_train.dtype, X_val.dtype, X_test.dtype))
print(" Max   : %d %d %d"%(X_train.max(), X_val.max(), X_test.max()) )
print(" Min   : %d %d %d"%(X_train.min(),X_val.min(), X_test.min()) )
plot_image( X_test[0].squeeze() )

An other way of representing labels is the one hot encoding form. MAny mathematical function can easly be generalized using this notation.
If they are 4 different labels, the sparse notation becomes:
   - label '0' --> 1 0 0 0
   - label '1' --> 0 1 0 0
   - label '2' --> 0 0 1 0
   - label '3' --> 0 0 0 1

This conversion can be done using a dedicated function:

In [ ]:
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_val_cat = tf.keras.utils.to_categorical(y_val)
y_test_cat = tf.keras.utils.to_categorical(y_test)

print(" Original (sparse) shape  : ", y_train.shape, y_val.shape, y_test.shape)
print(" Categorical shape  : ", y_train_cat.shape, y_val_cat.shape, y_test_cat.shape)
print(" Example : ", y_train[0,], " --> " ,y_train_cat[0,])

# Now changing name :
y_train = y_train_cat
y_val = y_val_cat
y_test = y_test_cat

### <span style="color:red"> **Question**
- What means the 9 for the last axis of y_train_cat, y_val_cat, y_test_cat?

> _**All done, go to the classification step!**_

# <span style="color:brown"> **Classification**

### Function for displaying reports

In [ ]:
# Usage example : ClassificationReport(model, X_test, y_test_cat , class_names)
def ClassificationReport(model, X_test, y_test_cat, labels_name):  
    # predict 
    print("Prediction on test images")
    predicted = model.predict(X_test)
    y_pred = np.argmax(predicted, axis=-1)
    y_test = np.argmax(y_test_cat, axis=-1)
    
    print("Classification report :\n%s\n"
         % (metrics.classification_report(y_test, y_pred)) )
    cm = metrics.confusion_matrix( y_test, y_pred )
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,  display_labels=labels_name)
    disp.plot()
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()

## <span style="color:brown"> **C - Classification with random forest**
    
First, we want to use a random forest to perform the classification. A random forest is an ensemble of decision trees. 
    
Each decision tree takes as input a feature vector. In our application, the feature vector is the grey-level values of the 32x32 (=1024 features) patch centered on each slice. 
    
During training, a matrix (X_train_crop) of size Nx1024 (N is the number of training images) is fed to the algorithm to build a decision tree. Learning the decision tree consists in finding the combination of features and cut-off values ($\alpha_i$) of each node such that the loss is minimal.
    
During test, the feature vector of one image is given to the learnt decision tree and goes through the first node. If the value of the feature (i.e. $f_5$ in the example) is higher than the cut-off value ($\alpha_0$), we proceed to the left branch, otherwise we proceed to the righ branch. This process is repeated until a leaf is reached, which will correspond to the predicted class.
    
A random forest is an ensemble of decision trees. M (M = 10 in the code below) decisions trees are learnt, each one from a subsample of the N training images. The final prediction is performed by a majority vote over the M predictions.
            
<center><img src="./figures/decision_tree.png" alt="Decision Tree" style="width: 50%"/></center>

In [ ]:
# here we prepare data for scikit-learn

# crop around the image center
crop_min = 32-16
crop_max = 32+16 +1 # '+1' is for python indexes : [0:10] corresponds to indexes 0 to 9 

# flattening for scikit-learn (ie a 64x64 image -> a vector of 4096 elements)
X_train_crop = []
for i in range(0, X_train.shape[0]):
    X_train_crop.append( X_train[i,crop_min:crop_max,crop_min:crop_max, 0].flatten() )
X_train_crop = np.asarray( X_train_crop )

X_val_crop = []
for i in range(0, X_val.shape[0]):
    X_val_crop.append( X_val[i,crop_min:crop_max,crop_min:crop_max, 0].flatten() )
X_val_crop = np.asarray( X_val_crop )

X_test_crop = []
for i in range(0, X_test.shape[0]):
    X_test_crop.append( X_test[i,crop_min:crop_max,crop_min:crop_max, 0].flatten() )
X_test_crop = np.asarray( X_test_crop )

print(" X_train_crop shape : ", X_train_crop.shape )
print(" y_train      shape : ", y_train.shape )
print(" X_val_crop   shape : ", X_val_crop.shape )
print(" y_val        shape : ", y_val.shape )
print(" X_test_crop  shape : ", X_test_crop.shape )
print(" y_test       shape : ", y_test.shape )

In [ ]:
from sklearn import ensemble
classifier = ensemble.RandomForestClassifier(n_estimators=10, max_depth=4, max_features ='auto', n_jobs=4, verbose=1)
#classifier = ensemble.GradientBoostingClassifier(n_estimators=10, max_depth=5, max_features ='auto', verbose=1) # can be very long !

print("Train model")
classifier.fit(X_train_crop, y_train)

print("Compute predictions")
y_pred = classifier.predict(X_test_crop)

In [ ]:
# evaluation
print("Classification report for classifier Random Forest :\n%s\n"
         % (metrics.classification_report(np.argmax(y_test, axis=-1), np.argmax(y_pred, axis=-1) )) )
cm = metrics.confusion_matrix( np.argmax(y_test, axis=-1), np.argmax(y_pred, axis=-1) )
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,  display_labels=class_names)
disp.plot()
disp.figure_.suptitle("Confusion Matrix")
plt.show()

### <span style="color:red"> **Questions**
- Modify the classification parameters to improve the accuracy. Objective : accuracy > 95%
- What do you think of the level of performance achieved?
- Try to test other classification approaches from [scikit-learn](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)

## <span style="color:brown"> **D - Classification with Networks**


### <span style="color:brown"> **D1 - Multi-Layer Perceptron**

In order to enter step by step into the world of deep learning, we will first look at the problem of classification of the [IXI dataset](https://brain-development.org/ixi-dataset/) using one of the simple network dedicated to such task, i.e. the multi-layer perceptron (also called a fully connected network - FCN). The corresponding architecture is provided below.

<center><img src="./figures/architecture_fcn_en.png" alt="Architecture FCN" style="width: 50%"/></center>

The execution of the cell below will allow you to load in memory the functions (actions) and constants that you will use afterwards. 

In [ ]:
# Declaration of functions that will be used from the standard  tensorflow library
from tensorflow.keras import optimizers
from tensorflow import losses

# Definition of constants
input_shape = (desired_height,desired_width,1)
input_shape_rgb = (desired_height,desired_width,3)
num_classes = 9

<span style="color:#ff7800"><b>Comment</b>: You can play with the value of some typical parameters by executing the cell below. During your session, feel free to go back to this cell, change some values and re-run the code from this cell to see the impact on the results.</span>

In [ ]:
# Definition of variables that you will have to play with during the session to study their influence

# Variables related to the FCN architecture
nb_mlp_neurons = 16    # Number of neurons for one layer

# Variables related to the optimization process
batch_size    = 16     # Number of images for each batch during the training process 
nb_epochs     = 5      # number of epochs used during the training process
learning_rate = 0.001  # learning rate used during the training process

Run the cell below to create a FCN (Fully Connected Network) model using a sequential declaration allowed by the tensorflow library

In [ ]:
# This part of the code allows you to free up memory space 
if 'model' in locals(): 
    print("deleting model")
    del model

# Define a FCN model from sequential declaration
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(nb_mlp_neurons, activation='relu'),  # a fully-connected layer with nb_mlp_neurons hidden units
  tf.keras.layers.Dense(num_classes, activation = "softmax")
])

# Display the network structure on the console
model.summary()

### <span style="color:red"> **Questions**
- Analyze the description of the network that appears above and try to make the link with the diagram of the fully connected network given on the figure above.    
- How many parameters does this network have in total ?
- Can you explain the 153 parameters that appear for the last layer (named dense_1) ?

It's time to start your first training by running the following cell !

In [ ]:
# Compile (define the optimization scheme) the model to be ready for training
model.compile(
  optimizer=optimizers.SGD(learning_rate=learning_rate),
  #optimizer=optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999),
  
  loss=losses.CategoricalCrossentropy(),
  #loss=losses.CategoricalHinge(),
  metrics=['accuracy'])

# Train the model
model.fit(
  X_train, y_train_cat,
  validation_data=(X_val, y_val_cat),
  batch_size=batch_size,  
  epochs=nb_epochs
)

<span style="color:#ff7800"><b>Comment</b>: During a training session, two sub-databases are used. The images belonging to the training sub-database (train) are directly used to update the network weights being optimised. The images belonging to the validation sub-database (valid) are used to evaluate the performance of the network on "neutral" (unused) images during optimisation. In the present configuration, the weights obtained at the end of the training process will be taken as the optimal ones.</span>

Once training is complete, you will apply the network to a sub-database called the test database. This database was never seen before by the network and will allow you to evaluate the performance of your network in an unbiased way. Ideally, we would like these results to be close to those obtained on the validation database.

Run the cell below to assess the performance of the previously learned network on the test database.

In [ ]:
# Compute the overall accuracy of the model on the test dataset 
test_loss, test_accuray = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', test_accuray)

# Compute and display standart classication metrics, i.e. precision, recall, f1-score, support and confusion matrix 
ClassificationReport(model, X_test, y_test, class_names)

### <span style="color:red"> **Questions**
- Now that you have completed your first learning from scratch, go back to the cell that allows you to modify the network parameters in order to perform new learning to see the influence of certain parameters. For example, you can play on the following parameters: <em>nb_epochs</em>, <em>learning_rate</em> et <em>nb_mlp_neurons</em>. To explore efficiently those parameters, multiply or divide by 2 or 10 their values.
- Add an additional fully connected layers by adding "tf.keras.layers.Dense()" functions in the sequential declaration of the model.
- Try to get the best possible performance (ie accuracy > 97% ).
- Compare the scores between the FCN and the random forest model.

### <span style="color:brown"> **D2 - Convolutional Neural Network (CNN)**

In order to try to improve the results, we are now looking at convolutional neural networks, of which an example architecture is provided below.

<center><img src="./figures/architecture_cnn_en.png" alt="Architecture CNN" style="width: 90%"/></center>


<span style="color:#ff7800"><b>Comment</b>: The interest of these kinds of networks is to add a succession of convolution layers in order to create an efficient (discriminating) representation of the data before the classification stage. These convolution layers also allow to significantly reduce the number of network parameters, which is a determining factor in the perspective of deploying these solutions in clinical routine (the fewer the parameters, the faster the network will run).</span>

<span style="color:#ff7800"><b>Comment</b>: You can play with the value of some typical parameters by executing the cell below. During your session, feel free to go back to this cell, change some values and re-run the code from this cell to see the impact on the results.</span>

In [ ]:
# Definition of variables that you will have to play with during the session to study their influence

# Variables related to the FCN architecture
nb_mlp_neurons = 16    # Number of neurons for one layer
nb_cnn_neurons = 16    # number of feature maps

# Variables related to the optimization process
batch_size    = 16     # Number of images for each batch during the training process 
nb_epochs     = 5      # number of epochs used during the training process
learning_rate = 0.001  # learning rate used during the training process

Run the cell below to create a CNN (Convolutional Neural Network) model using a sequential declaration allowed by the tensorflow library

In [ ]:
# This part of the code allows you to free up memory space 
if 'model' in locals(): 
    print("deleting model")
    del model
    
# Define a CNN model from sequential declaration    
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.Conv2D(nb_cnn_neurons//2, 3, activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(nb_cnn_neurons, 3, activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(nb_cnn_neurons*2, 3, activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(nb_mlp_neurons, activation='relu'),  #128
  tf.keras.layers.Dense(num_classes, activation = "softmax")
])

# Display the network structure
model.summary()

### <span style="color:red"> **Questions**
- Analyze the description of the network that appears after the execution of the cell above and try to make the link with the CNN diagram given above.    
- How many parameters does this network have in total ?
- Compare this value with the one of the FCN studied above.
- How can you explain this favourable difference ?

Run the cell below to train the CNN model

In [ ]:
# Compile (define the optimization scheme) the model to be ready for training
model.compile(
  optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
  loss=tf.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

# Train the model
model.fit(
  X_train, y_train,
  validation_data=(X_val, y_val),
  batch_size=batch_size,  
  epochs=nb_epochs
)

Run the cell below to evaluate the performance of the model on the test dataset.

In [ ]:
# Compute the overall accuracy of the model on the test dataset 
test_loss, test_accuray = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', test_accuray)

# Compute and display standart classication metrics, i.e. precision, recall, f1-score, support and confusion matrix 
ClassificationReport(model, X_test, y_test, class_names)

### <span style="color:red"> **Questions**
- From the cells allowing to modify the network architecture as well as the optimisation parameters, try to get the best possible performance (if necessary...).
- Compare the scores between the CNN and FCN models.
- Is there a big difference? What conclusions can you draw from this ?

### <span style="color:brown"> **D3 - Residual deep network and transfer learning (fine tuning)**
    
We will now conclude this hands-on session by working on one of the most powerful networks currently used for image classification, namely the resnet model whose architecture is recalled below. 

<center><img src="./figures/architecture_resnet_en.png" alt="Architecture ResNet" style="width: 60%"/></center>


This architecture is very deep and potentially challenging to train. In order to be efficient, the use of such an architecture for a given application is usually done through a particular strategy, called transfer learning.

The idea behind transfer learning is to reuse the weights of a part of the network that have already been learned from large databases, such as imagenet. This section of the architecture, named base, corresponds to the set of layers before the classification part. Thus, for a given application, the base part of the resnet with the associated pre-trained weights is selected first. A dedicated classification subnetwork (i.e. a fully connected network) is then added at the end to form a new architecture. During the training phase, only the weights of the network's head (i.e. the classification part) are optimized, making the learning process faster.

In the remainder of this hands on session, you will create such a resnet-based architecture using transfer learning strategy.

<span style="color:#ff7800"><b>Comment</b>: You can play with the value of some typical parameters by executing the cell below. During your session, feel free to go back to this cell, change some values and re-run the code from this cell to see the impact on the results.</span>

In [ ]:
# Definition of variables that you will have to play with during the session to study their influence

# Variables related to the FCN architecture
nb_mlp_neurons = 16    # Number of neurons for one layer

# Variables related to the optimization process
batch_size  = 16       # Number of images for each batch during the training process 
nb_epochs   = 5          # number of epochs used during the training process
learning_rate = 0.001  # learning rate used during the training process

Run the cell below to create a resnet-based model using a predifined structure from the tensorflow library.
This code is explained in the figure below

In [ ]:
# This part of the code allows you to free up memory space 
if 'model' in locals(): 
    print("deleting model")
    del model

# transfert learning / fine tuning head (classification part) only
print(" ResNET50 network is trained with RGB natural images, so we use the RGB train and test datasets.")
    
# Define input and preprocessed them to fit the conditions used for the predifined resent model 
inputs = tf.keras.layers.Input(shape=input_shape_rgb)
x = tf.cast(inputs, tf.float32)
outputs_preproc = tf.keras.applications.resnet_v2.preprocess_input(x)  # pixels values are changed to [-1;1]

# Define the ResNet architecture and dowload weights trained on imagenet
baseModel = tf.keras.applications.ResNet50V2(
    weights="imagenet", # use "imagenet" to use imagenet resnet weights, or None (!! no " ") for random initialization of network parameters
    include_top=False,  # don't include the fully-connected layers (classification part) at the top of ResNet
    input_tensor=tf.keras.layers.Input(shape=input_shape_rgb))

# Link outputs of preprocessing as intput of baseModel
outputs_base = baseModel(outputs_preproc)

# Create the model of preproc + resnet without 'top' or 'head'
baseModel_preproc = tf.keras.Model(inputs=[inputs], outputs=[outputs_base]) 
    
# Define the head model i.e. the part that is flattenning alls features extracted before and then classify the data
# and which is fully connected (MLP...) in order to obtain the classification
headModel = baseModel_preproc.output
headModel = tf.keras.layers.Flatten(name = "flatten")(headModel)
headModel = tf.keras.layers.Dense(nb_mlp_neurons, activation = "relu")(headModel)
outputs_head = tf.keras.layers.Dense(num_classes, activation = "softmax")(headModel)

# Create the final resnet-base model
model = tf.keras.Model(inputs=[inputs], outputs=outputs_head)

# allow baseModel weights to be trainable ? (if True, the training step will be long i.e. 1 to 3 minutes per epoch)
baseModel.trainable = False

# Display the network structure on the console
model.summary()

# uncomment to see ResNet50 architecture (it is a deep network :p )
#baseModel.summary()


<center><img src="./figures/ResNet_TransferLearning.png" alt="ResNet Transfer Learning schme" style="width: 60%" /></center>

### <span style="color:red"> **Questions**
- Analyze the description of the network that appears after the execution of the cell above and try to make the link with the diagram given above.    
- How many parameters does this network have in total ?
- Compare this value with the one of the CNN studied above.
- Where do the differences come from?

In [ ]:
# Compile (define the optimization scheme) the model to be ready for training
model.compile(
  optimizer=optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999),
  loss=tf.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

# Train the model
model.fit(
  X_train_rgb, y_train,
  validation_data=(X_val_rgb, y_val),
  batch_size=batch_size,  
  epochs=nb_epochs
)

Run the cell below to evaluate the performance of the model on the test dataset.

In [ ]:
# Compute the overall accuracy of the model on the test dataset 
test_loss, test_accuray = model.evaluate(X_test_rgb, y_test, verbose=1)
print('Test accuracy:', test_accuray)

# Compute and display standart classication metrics, i.e. precision, recall, f1-score, support and confusion matrix 
ClassificationReport(model, X_test_rgb, y_test, class_names)

### <span style="color:red"> **Questions**
- From the cells allowing to modify the network architecture as well as the optimisation parameters, try to get the best possible performance.
- The training time is longer, why ? 
- Is the accuracy better than previous and smaller network? Have a look at the number of trainable parameters...
- [long test] Is the accuracy better if all weights of the network can be trained ? (i.e. including the ones of baseModel)
- [very long test] How to assess the role of the initial weights coming from imagenet ? (by the way : what is imagenet?)

In [ ]:
# Restore the best weights according val_loss
# don't execute this cell! 
#   copy and adapt it in the previous cells...

from tensorflow.keras.callbacks import EarlyStopping

callback_earlystopping = EarlyStopping(
    monitor='val_loss', #'val_loss'
    mode ='auto', # use 'min' or 'auto' if val_loss
    patience=NBPATIENCE_EPOCHS,
    restore_best_weights=True  # at the end of fitting, restore best model weights 
)

model.fit(
  X_train_rgb, y_train,  # or X_train, y_train,
  validation_data=(X_val_rgb, y_val), # or (X_val, y_val)
  batch_size=batch_size,  
  epochs=nb_epochs,
  callbacks=[callback_earlystopping]
)